In [1]:
##! SKIP THIS if on Google Colaboratory
%load_ext notexbook
%texify

# Fast Gradient Sign Attack

This notebook showcases how to carry out a **Fast Gradient Sign Attack** (`FGSA`) to a pretrained model. 

**Note** This notebook has been adapted from the [FSGM Tutorial](https://pytorch.org/tutorials/beginner/fgsm_tutorial.html) by _Nathan Inkawhich_ `@inkawhich` available on the official [PyTorch Documentation](https://pytorch.org/docs/stable/index.html)

(_from the original notebook_)
> This tutorial will raise your awareness to the security vulnerabilities 
> of ML models, and will give insight into the hot topic of adversarial machine learning. 
> 
> You may be surprised to find that adding **imperceptible perturbations** to an image *can* cause 
> drastically different model performance.
> `[...]`
>
> Specifically we will use one of the first and most popular attack methods, the _Fast Gradient Sign Attack_
> (`FGSM`), to fool an `MNIST` classifier.


## Threats to Models

There are several kinds of assumptions of the attacker’s knowledge, two of which are: **white-box** and **black-box**. 

- A *white-box* attack assumes the attacker has full knowledge and access to the model, including
architecture, inputs, outputs, and weights. 
- A *black-box* attack assumes the attacker only has access to the inputs and outputs of the model, and knows nothing about the underlying architecture or weights. 

There are also several types of goals, including **misclassification** and
**source/target misclassification**. 

A goal of *misclassification* means the adversary only wants the output classification to be wrong but does
not care what the new classification is. 

A *source/target misclassification* means the adversary wants to alter an image that is originally of a specific source class so that it is classified as a specific target class.

## Fast Gradient Sign Attack

*Fast Gradient Sign Attack (FGSM)* and is described by _Goodfellow et. al._ in 
[Explaining and Harnessing Adversarial Examples](https://arxiv.org/abs/1412.6572). 

The attack is remarkably powerful, and yet intuitive. 

It is designed to attack neural networks by leveraging the way they learn: **gradients**. 

The idea is simple: 

> rather than working to minimize the loss by adjusting the weights based on the backpropagated gradients,
> the attack **adjusts** the input data to maximize the loss based on the same backpropagated gradients. 

In other words, the attack uses the gradient of the loss w.r.t the input data, then adjusts the input data to maximize the loss.

_(from the original paper)_

![fgsm panda attack](https://pytorch.org/tutorials/_images/fgsm_panda_image.png)

**TLDR;** Just perturbe the input data with some small change that would work in an **adversary** fashion (wrt. the optimisation process) that follows the **direction of the gradient** (i.e. $sign(\nabla_{x} J(\mathbf{\theta}, \mathbf{x}, y))$ )

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# NOTE: This is a hack to get around "User-agent" limitations when downloading MNIST datasets
#       see, https://github.com/pytorch/vision/issues/3497 for more information
from six.moves import urllib

opener = urllib.request.build_opener()
opener.addheaders = [("User-agent", "Mozilla/5.0")]
urllib.request.install_opener(opener)

In [ ]:
from pathlib import Path
import os 

DATA_FOLDER = Path(os.path.join(os.path.abspath(os.path.curdir), "..", "..")) / "data"

In [ ]:
# Reproducibility Settings

import numpy as np

SEED = 123456
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

#### `LeNet` Model

In [ ]:
# LeNet Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Initialise pre-trained model (and move it to available device)

In [ ]:
PRETRAINED_MODEL_WEIGHTS = "lenet_mnist_model.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the network
model = Net().to(device)

# Load the pretrained model
model.load_state_dict(torch.load(PRETRAINED_MODEL_WEIGHTS, map_location=device))

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

### Download MNIST Dataset

In [ ]:
# MNIST Test dataset and dataloader declaration
mnist_test = datasets.MNIST(root=DATA_FOLDER, train=False, download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=1, shuffle=False)

### Before the Attack

Before carrying out the attack, let's see how well the model classify the digits in the test set

In [ ]:
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

y_preds, y_true = list(), list()
with torch.no_grad():  # extra, as model is eval mode anyway
    for (image, target) in tqdm(test_loader):
        image, target = image.to(device), target.to(device)
        out = model(image)
        _, preds = torch.max(out, 1)
        y_preds.append(preds.detach().cpu().numpy())
        y_true.append(target.detach().cpu().numpy())
    y_preds = np.hstack(y_preds)
    y_true = np.hstack(y_true)
    
    print(f"Pre-Trained Model ACC: {accuracy_score(y_true, y_preds)}")

### `FSGM` Attack

Now, we can define the function that creates the adversarial examples by
perturbing the original inputs. 

In [ ]:
# FGSM attack code
def fgsm_attack(image: torch.Tensor, epsilon: float, data_gradient: torch.Tensor) -> torch.Tensor:
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_gradient.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + (epsilon * sign_data_grad)
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)  # normalise in [0, 1] to make it an actual image
    # Return the perturbed image
    return perturbed_image

Last but not least: the **test function**

In [ ]:
def test(model, device, loader, epsilon):
    # from https://github.com/pytorch/tutorials/blob/master/beginner_source/fgsm_tutorial.py
    
    # Accuracy counter
    correct = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in tqdm(test_loader, desc=f"Running Attack on Batches with ε={ε}"):

        # Send the data and label to the device
        data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]  # get the index of the max log-probability

        # If the initial prediction is wrong, don't bother attacking, just move on
        if init_pred.item() != target.item():
            continue

        # Calculate the loss
        loss = F.nll_loss(output, target)

        # Zero all existing gradients
        model.zero_grad()

        # Calculate gradients of model in backward pass
        loss.backward()

        # Collect datagrad
        data_grad = data.grad.data

        # Call FGSM Attack
        perturbed_data = fgsm_attack(data, epsilon, data_grad)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1]  # get the index of the max log-probability
        if final_pred.item() == target.item():
            correct += 1
        else:
            # Save some adv examples for visualization later
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))

    # Calculate final accuracy for this epsilon
    final_acc = correct / float(len(test_loader))
    print(
        "Epsilon: {}\tTest Accuracy = {} / {} = {}".format(
            epsilon, correct, len(test_loader), final_acc
        )
    )

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

### Run the Attack

In [ ]:
ε = 0.05

In [ ]:
acc, adv_examples = test(model, device, test_loader, ε)

Now let's see how the perturbed images look like: 

In [ ]:
fig, axes = plt.subplots(1, len(adv_examples), figsize=(8, 10))
plt.xticks([], [])
plt.yticks([], [])
for j, (orig_pred, adv_pred, adv_example) in enumerate(adv_examples):
    if j == 0:
        axes[j].set_ylabel(f"ε: {ε}", fontsize=14)
    axes[j].set_title("{} -> {}".format(orig_pred, adv_pred))
    axes[j].imshow(adv_example, cmap="gray")
plt.tight_layout()
plt.show()

### Exercise:

Now the question is: how much degradation in performance we have as soon as we keep incrementing the value of ε?

What we should expect: 
- the bigger ε, the worse the accuracy
- the bigger ε, the more "discoverable" the perturbation becomes
    - so that it's evident that an attack has been launched

In [ ]:
EPSILONS = [0.5, .06, .1, .15, .2, .25, .3]

accuracies = [acc]
adv_examples_map = {0.5: adv_examples}

# Run test for each epsilon
for ε in EPSILONS[1:]:
    # your code here

### Results

1. Let's print the accuracy values for each corresponding ε value

In [ ]:
accuracies

In [ ]:
plt.figure(figsize=(5, 5))

# your code here: plot Accuracies vs EPSILONS
plt.plot(EPSILONS, accuracies)
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, 0.35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

2. Visualise Generated Adversarial Examples

In [ ]:
# Plot several examples of adversarial samples at each epsilon

for ε in EPSILONS:
    fig, axes = plt.subplots(1, len(adv_examples_map[ε]), figsize=(8, 10))
    plt.xticks([], [])
    plt.yticks([], [])
    for j, (orig_pred, adv_pred, adv_example) in enumerate(adv_examples_map[ε]):
        if j == 0:
            axes[j].set_ylabel(f"ε: {ε}", fontsize=14)
        axes[j].set_title("{} -> {}".format(orig_pred, adv_pred))
        axes[j].imshow(adv_example, cmap="gray")
    plt.tight_layout()
    plt.show()